In [1]:
import pycromanager as pyc

In [2]:
from pycromanager import Core,Studio

In [3]:
core = Core()

In [4]:
core.get_position()

-61.6

In [5]:
brushless = "KBD101_Thor_Rotation"
core.get_position(brushless)

32.9145

In [6]:
def ppm_to_thor(angle):
    return (-2*angle + 276)

def thor_to_ppm(kinesis_pos):
    return (276 - kinesis_pos) / 2

In [7]:
thor_to_ppm(33)

121.5

In [33]:
def set_angle(theta):
    theta = ppm_to_thor(theta)
    core.set_position(brushless,theta)
    core.wait_for_device(brushless)
set_angle(0)

In [20]:
from typing import Tuple,Dict,Any
import numpy as np
def snap_image() -> Tuple[np.ndarray, Dict[str, Any]]:
    core.snap_image()
    img = core.get_image()
    tags = {
        "Width": core.get_image_width(),
        "Height": core.get_image_height(),
        #"PixelType": core.get_pixel_type(),
        #"BytesPerPixel": core.get_bytes_per_pixel(),
        #"BitDepth": core.get_bitdepth(),
    }
    # Handle color cameras
    camera = core.get_property('Core','Camera')
    if camera in ["QCamera", "MicroPublisher6"]:
        if core.get_property(camera,'Color') == "ON":
            img = np.reshape(img, (tags["Height"], tags["Width"], 4))
            img = img[:, :, 0:3]  # Remove alpha
            img = np.flip(img, 2)  # Flip channels
        else:
            img = np.reshape(img, (tags["Height"], tags["Width"]))
    else:
        img = np.reshape(img, (tags["Height"], tags["Width"]))
    return img, tags


In [21]:
img,tags = snap_image()

In [22]:
import matplotlib.pyplot as plt

In [28]:
#plt.imshow(img,cmap='gray')
#plt.colorbar()

In [29]:
img.dtype

dtype('uint16')

In [30]:
img.shape

(2200, 2688)

In [31]:
import tifffile as tf

# pure_calibration

In [38]:
import os
os.getcwd()

'C:\\Users\\lociuser\\Codes\\smartpath'

In [51]:
from fastprogress import progress_bar

In [54]:
for k in progress_bar(range(0,360),total=360):
    set_angle(k)
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'D:\UserData\MichealNelson\debeyer_data_calibration_20xPolp5NA\Calibration_angle{k:03}.tif'
    tf.imwrite(filename,img.astype(np.uint16))    
    with open('log.txt','a') as fid:
        fid.write(str([k, theta, img.mean()]))

# 3 mins for full 360 deg calibration with 150 ms exposure